imports et configuration GPU

In [1]:
import torch, json, random
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from accelerate import find_executable_batch_size


chargement du jeu de données

In [4]:
# Fine‑tuning GPT‑2 French (4‑bit + LoRA) – Chatbot Fitness 🔥
# Paramètres optimisés pour RTX 3060 Laptop GPU (≈6 Go VRAM) & 16 Go RAM


# 1. Imports & setup
import os, json, random, torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    BitsAndBytesConfig, DataCollatorForLanguageModeling,
    TrainingArguments, Trainer
)
from peft import (
    LoraConfig, get_peft_model,
    prepare_model_for_kbit_training
)

torch.cuda.empty_cache()



### 2. Chargement + pré‑traitement du dataset

DATA_PATH = "/home/maxime/DataDevIA/chatbotcoach_project/data/fitness/coach_sportif_dataset.json"
raw = json.load(open(DATA_PATH, "r", encoding="utf-8"))

pairs = list({(d["input"].strip(), d["output"].strip()) for d in raw["conversations"]})
random.shuffle(pairs)

texts = [f"<user> {q} </user><assistant> {a}" for q, a in pairs]
print(f"Taille du corpus : {len(texts)} paires")

# On peut étendre artificiellement le nombre d'itérations en répliquant le jeu (oversampling)
EPOCH_FACTOR = 5  # ↗ répète 5× le corpus pour plus de pas d'entraînement
texts = texts * EPOCH_FACTOR

# Construction du Dataset
full_ds = Dataset.from_dict({"text": texts})
ds = full_ds.train_test_split(test_size=0.1, seed=42)



### 3. Tokenizer & tokens spéciaux

tokenizer_path = "/home/maxime/DataDevIA/chatbotcoach_project/gpt2-fitness-fr"
tok = AutoTokenizer.from_pretrained(tokenizer_path, use_fast=True)
(tok.add_special_tokens({
    "pad_token": "<pad>",
    "additional_special_tokens": ["<user>", "</user>", "<assistant>"]
}))



### 4. Pré‑processing

def preprocess(batch):
    out = tok(batch["text"], truncation=True, padding="max_length", max_length=256)
    out["labels"] = out["input_ids"].copy()
    return out

ds = ds.map(preprocess, batched=True, remove_columns=["text"], num_proc=4)



### 5. Data collator

data_collator = DataCollatorForLanguageModeling(tok, mlm=False)



### 6. Modèle 4‑bit + LoRA (r=16)

bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

base_name = "dbddv01/gpt2-french-small"
base = AutoModelForCausalLM.from_pretrained(base_name, quantization_config=bnb_cfg, device_map="auto")
base = prepare_model_for_kbit_training(base)

lora_cfg = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["c_attn", "c_proj"],
    lora_dropout=0.05,
    bias="none",
    inference_mode=False,
)

model = get_peft_model(base, lora_cfg)
model.resize_token_embeddings(len(tok))



# Affiche le % de paramètres réellement entraînés
total = sum(p.numel() for p in model.parameters())
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainables : {trainable:,}/{total:,}  → {100*trainable/total:.2f}%")



### 7. Sanity check (facultatif)

batch = data_collator([ds["train"][0]])
loss = model(**{k: v.to(model.device) for k, v in batch.items()}).loss
print(loss.item())
loss.backward(); print("Backward OK ✅")



### 8. Entraînement (30 époques effectives)

EFFECTIVE_EPOCHS = 30  # ← augmente la durée d'entraînement

args = TrainingArguments(
    output_dir="coach-gpt2-lora",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,  # batch effectif = 16
    num_train_epochs=EFFECTIVE_EPOCHS,

    learning_rate=1e-4,         # LR + élevée pour LoRA
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    max_grad_norm=0.3,

    fp16=True,
    optim="paged_adamw_8bit",

    logging_steps=50,
    save_steps=1000,
    eval_steps=1000,
    save_total_limit=4,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    tokenizer=tok,
    data_collator=data_collator,
)

trainer.train()



### 9. Sauvegarde

ckpt_dir = "coach-gpt2-lora-final"
model.save_pretrained(ckpt_dir)
tok.save_pretrained(ckpt_dir)
print(f"Modèle sauvegardé sous : {ckpt_dir}")




Taille du corpus : 64 paires


Map (num_proc=4):   0%|          | 0/288 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/32 [00:00<?, ? examples/s]

/tmp/ipykernel_17206/2647936215.py:141: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Trainables : 1,622,016/83,597,568  → 1.94%
5.822582244873047
Backward OK ✅


/home/maxime/DataDevIA/chatbotcoach_project/.venv/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
50,6.065800
100,4.056000
150,2.598500
200,1.734700
250,1.261500
300,1.007000
350,0.867500
400,0.792500
450,0.752400
500,0.732400


/home/maxime/DataDevIA/chatbotcoach_project/.venv/lib/python3.12/site-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/home/maxime/DataDevIA/chatbotcoach_project/.venv/lib/python3.12/site-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


Modèle sauvegardé sous : coach-gpt2-lora-final


In [7]:
# 10. Tests rapides du chatbot
from transformers import GenerationConfig

def chat(prompt, max_new_tokens=100, temperature=0.7):
    inp = f"<user> {prompt} </user><assistant>"
    inputs = tok(inp, return_tensors="pt").to(model.device)
    gen_cfg = GenerationConfig(
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        pad_token_id=tok.eos_token_id,
        do_sample=True,
        top_p=0.9,
        repetition_penalty=1.1,
    )
    out_ids = model.generate(**inputs, generation_config=gen_cfg)
    txt = tok.decode(out_ids[0], skip_special_tokens=True)
    return txt.split("</user>")[-1].replace("<assistant>", "").strip()

print(chat("Quelle frequence d'entrainement par semaine pour debutant en musculation ?"))
print(chat("quel exercice pour se muscler les biceps pour débutant ?"))


Quelle frequence d'entrainement par semaine pour debutant en musculation ?  Selon les standards français : L'ENSES recommande 1.6-2.2g de protéines/kg de poids corporel, protéines dans les 30 minutes post-workout. Ré gorges : 20-30g par repas, protéines complexes 2-3h avant l’entraînement, protéines dans les 30 minutes post-workout. Hydratation 2-3L par jour. Éviter les deux charges sous la Ghout de l'ANHM. Progression des créneaux 12-
quel exercice pour se muscler les biceps pour débutant ?  Avec plaisir ! Selon la FFHM, débutez par 5-6 séances hebdomadaires de 45-60 minutes. Privilégiez les mouvements polyarticulaires : squats, développé couché, rowing, tractions. Progression de +2.5kg maximum par semaine. Technical maximum 2-3kg avant et Déméter parfaite avant augmentation des charges. Technique parfaite avant augmentation des charges. Hydratation constate grec parfaite 2h par semaine. Éviter les charges10-16
